In [21]:
pip install sentence-transformers

  Using cached sentence_transformers-2.2.2-py3-none-any.whl
Note: you may need to restart the kernel to use updated packages.


In [24]:
pip install chromadb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 1.4 MB/s eta 0:00:00a 0:00:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.9/45.9 kB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 151.7/151.7 kB 3.5 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 12.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 15.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 181.3/181.3 kB 21.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 20.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 502.4/502.4 kB 11.7 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 34.5 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 699.4/699.4 kB 3.8 MB/s eta 0:00:0000:0100:01m
   ━━━━━━━━━━━━

In [26]:
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter 
from langchain.embeddings import SentenceTransformerEmbeddings 
from langchain.vectorstores import Chroma
from langchain.embeddings import OpenAIEmbeddings
from langchain.retrievers.multi_query import MultiQueryRetriever
from langchain.chains import RetrievalQA
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM 
from transformers import pipeline
from langchain.llms import HuggingFacePipeline
from langchain.embeddings import HuggingFaceEmbeddings
from transformers import pipeline, AutoModelForCausalLM
import torch


#Loader
loader = PyPDFLoader("docs/carry.pdf")
pages = loader.load_and_split()



MODEL = 'beomi/KoAlpaca-Polyglot-5.8B'
device ="cuda"

tokenizer = AutoTokenizer.from_pretrained(MODEL)

base_model = AutoModelForCausalLM.from_pretrained(
    MODEL,
    torch_dtype=torch.float16,
    low_cpu_mem_usage=True,
).to(device=device, non_blocking=True)
base_model.eval()


#Split
text_splitter = RecursiveCharacterTextSplitter(
    # Set a really small chunk size, just to show.
    chunk_size = 300,
    chunk_overlap  = 20,
    length_function = len,
    is_separator_regex = False,
)
texts = text_splitter.split_documents(pages)

print(texts)


#Embedding
embeddings_model = HuggingFaceEmbeddings(
    model_name="jhgan/ko-sroberta-multitask", encode_kwargs={'normalize_embeddings': True}
)


# load it into Chroma
db = Chroma.from_documents(texts, embeddings_model)


def llm_pipeline():
    pipe = pipeline(
        'text2text-generation',
        model = base_model,
        tokenizer = tokenizer,
        max_length = 256,
        do_sample = True,
        temperature = 0.3,
        top_p= 0.95,
        device=device
    )
    local_llm = HuggingFacePipeline(pipeline=pipe)
    return local_llm


#Question

llm = llm_pipeline()
retriver_from_llm = MultiQueryRetriever.from_llm(
    retriever=db.as_retriever(), llm=llm
)





Loading checkpoint shards:   0%|          | 0/13 [00:00<?, ?it/s]

[Document(page_content='carryprotocol.io \xa0 4 \xa0\n \xa0\n 1.         Executive Summary \xa0\n \xa0\n 캐리 프로토콜은 블록체인 기술을 기반으로 오프라인 상점과 소비자를 연결하는 플랫폼이다. \xa0\n 캐리 프로토콜은 온라인 커머스와 모바일 커머스의 빠른 발전에도 불구하고 사람들이 여전히 \xa0\n 오프라인에서 대부분의 소비를 하고 있다는 것에 주목하였다. 특히 오프라인 커머스 시장은 2.7 경원이라는 \xa0\n 매력적인 시장 크기를 자랑하지만, 상당히 기술적으로 낙후되어 있어 개선해야 할 점들이 다수 존재한다.', metadata={'source': 'docs/carry.pdf', 'page': 0}), Document(page_content='이는 기술의 도입과 확산에 너무나 많은 노력과 시간을 요구하는 오프라인 시장의 특성에 기인한다. \xa0\n 낙후된 오프라인 커머스에는 세 가지 큰 문제점이 있다. 첫째, 상점마다 고객 정보를 따로 관리하거나 \xa0\n 전혀 관리하지 못하고 있어, 상점들은 그들의 고객을 전혀 이해하지 못하고 있다. 둘째, 소비자들은 \xa0\n 자신들의 결제 데이터를 통제하지 못하고 있는 반면, 거대 기업들은 이를 마음대로 이용하며 수익화하고 \xa0\n 있다. 셋째, 오프라인 커머스 광고는 기술적으로 매우 낙후되어 있어 불투명하고 광고 효과에 대한 추적도', metadata={'source': 'docs/carry.pdf', 'page': 0}), Document(page_content='어렵다. \xa0\n 캐리 프로토콜은 오프라인의 이런 문제점들을 해결하기 위해 다음과 같은 세 가지 미션을 제시한다. 1) \xa0\n 상점이 고객을 이해하고, 그들과 상호 소통할 수 있는 플랫폼을 제공한다. 2) 소비자가 자신의 결제 \xa0\n 데이터에 대한 통제권을 갖고 이를 수익화할 수 있도록 지원한다. 3) 신뢰할 수 있고, 높은 광고 효율의 \xa

The model 'GPTNeoXForCausalLM' is not supported for text2text-generation. Supported models are ['BartForConditionalGeneration', 'BigBirdPegasusForConditionalGeneration', 'BlenderbotForConditionalGeneration', 'BlenderbotSmallForConditionalGeneration', 'EncoderDecoderModel', 'FSMTForConditionalGeneration', 'GPTSanJapaneseForConditionalGeneration', 'LEDForConditionalGeneration', 'LongT5ForConditionalGeneration', 'M2M100ForConditionalGeneration', 'MarianMTModel', 'MBartForConditionalGeneration', 'MT5ForConditionalGeneration', 'MvpForConditionalGeneration', 'NllbMoeForConditionalGeneration', 'PegasusForConditionalGeneration', 'PegasusXForConditionalGeneration', 'PLBartForConditionalGeneration', 'ProphetNetForConditionalGeneration', 'SeamlessM4TForTextToText', 'SwitchTransformersForConditionalGeneration', 'T5ForConditionalGeneration', 'UMT5ForConditionalGeneration', 'XLMProphetNetForConditionalGeneration'].


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 1.5 MB/s eta 0:00:00a 0:00:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.4/60.4 kB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 11.4 MB/s eta 0:00:00 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 93.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.8/10.8 MB 46.1 MB/s eta 0:00:0000:010:01m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 36.4/36.4 MB 43.5 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 670.2/670.2 MB 7.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.2/302.2 kB 63.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.9/97.9 kB 24.6 MB/s eta 0:00:00
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125923 sha256=5bb4fcbc589740746551fe662476f126819067009eb09986dc3cf222e13d11de
  Stored in d

In [31]:

db = Chroma(persist_directory="db", embedding_function = embeddings_model)
retriever = db.as_retriever()
qa = RetrievalQA.from_chain_type(
    llm = llm,
    chain_type = "stuff",
    retriever = retriever,
    return_source_documents=True
)


In [32]:
question = "캐리프로토콜에 대해 설명해줘"
print(qa(question))


/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1473: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use and modify the model generation configuration (see https://huggingface.co/docs/transformers/generation_strategies#default-text-generation-configuration )
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


{'query': '캐리프로토콜에 대해 설명해줘', 'result': "Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.\n\n\n\nQuestion: 캐리프로토콜에 대해 설명해줘\nHelpful Answer:\n\nCarry Protocol은 인터넷 프로토콜입니다. 이 프로토콜은 인터넷에서 데이터를 주고 받는 데 사용되며, 웹 브라우저와 다른 웹 서버 간에 통신을 가능하게 합니다. Carry Protocol은 네스케이프(Netscape)에서 시작되었으며, 이후 윈도우 운영체제에 의해 지원되었습니다. 윈도우 98 서비스에서는 Carry Protocol을 사용하여 인터넷 연결을 설정할 수 있습니다. \n\nCarry Protocol은 일반적으로 웹 브라우저의 주소창에 URL을 입력함으로써", 'source_documents': []}
